In [1]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import xlrd
import time
from IPython.display import Audio

In [2]:
#finish indicator
wow = '../../Data/Wow.mp3'
finished=Audio(url = wow, autoplay = True) 

In [27]:
map = gpd.read_file('../../Data/NYC_Geo_prayer.json')

In [28]:
map=map[['postalCode','PO_NAME','borough','geometry']]

In [4]:
df=pd.read_csv('../../Data/Taxi Data/yellow_14/yellow_tripdata_2014-1.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13782492 entries, 0 to 13782491
Data columns (total 18 columns):
 #   Column               Dtype  
---  ------               -----  
 0   vendor_id            object 
 1    pickup_datetime     object 
 2    dropoff_datetime    object 
 3    passenger_count     int64  
 4    trip_distance       float64
 5    pickup_longitude    float64
 6    pickup_latitude     float64
 7    rate_code           int64  
 8    store_and_fwd_flag  object 
 9    dropoff_longitude   float64
 10   dropoff_latitude    float64
 11   payment_type        object 
 12   fare_amount         float64
 13   surcharge           float64
 14   mta_tax             float64
 15   tip_amount          float64
 16   tolls_amount        float64
 17   total_amount        float64
dtypes: float64(11), int64(2), object(5)
memory usage: 1.8+ GB


In [6]:
df2=df.head(100)

In [10]:
df2.columns

Index(['vendor_id', ' pickup_datetime', ' dropoff_datetime',
       ' passenger_count', ' trip_distance', ' pickup_longitude',
       ' pickup_latitude', ' rate_code', ' store_and_fwd_flag',
       ' dropoff_longitude', ' dropoff_latitude', ' payment_type',
       ' fare_amount', ' surcharge', ' mta_tax', ' tip_amount',
       ' tolls_amount', ' total_amount'],
      dtype='object')

In [ ]:
df2['start_lon']=df2.iloc[:,5]

In [ ]:
df2['start_lat']=df2.iloc[:,6]

In [15]:
start_time = time.time()

df['dropoff_geometry'] = df.apply(lambda x: Point((float(x[' dropoff_longitude']), 
                                             float(x[' dropoff_latitude']))), 
                                             axis=1)
df.head(3)

print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 817.3952524662018 seconds ---


In [16]:
df.head(3)

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount,pickup_geometry,dropoff_geometry
0,CMT,2014-01-09 20:45:25,2014-01-09 20:52:31,1,0.7,-73.994770,40.736828,1,N,-73.982227,40.731790,CRD,6.5,0.5,0.5,1.4,0.0,8.9,POINT (-73.99477 40.736828),POINT (-73.98222699999998 40.73179)
1,CMT,2014-01-09 20:46:12,2014-01-09 20:55:12,1,1.4,-73.982392,40.773382,1,N,-73.960449,40.763995,CRD,8.5,0.5,0.5,1.9,0.0,11.4,POINT (-73.982392 40.773382),POINT (-73.96044899999998 40.763995)
2,CMT,2014-01-09 20:44:47,2014-01-09 20:59:46,2,2.3,-73.988570,40.739406,1,N,-73.986626,40.765217,CRD,11.5,0.5,0.5,1.5,0.0,14.0,POINT (-73.98856999999998 40.739406),POINT (-73.986626 40.765217)


In [32]:
df2_start=df2[[' passenger_count',' trip_distance',' fare_amount',' mta_tax',' tip_amount',' tolls_amount','pickup_geometry']]

In [36]:
df2_end=df2[['dropoff_geometry']]

In [40]:
start_time = time.time()
df2_start=gpd.GeoDataFrame(df2_start,
                     crs=map.crs,
                     geometry=df2_start['pickup_geometry'])
print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 112.19443106651306 seconds ---


In [42]:
start_time = time.time()

geo_processing = gpd.sjoin(df2_start, map, op = 'within')

print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 1175.6654407978058 seconds ---


In [47]:
geo_processing

,passenger_count,trip_distance,fare_amount,mta_tax,tip_amount,tolls_amount,pickup_geometry,geometry,index_right,postalCode,PO_NAME,borough
0,1,0.70,6.5,0.5,1.4,0.0,POINT (-73.99477 40.736828),POINT (-73.99477 40.73683),114,10011,New York,Manhattan
24,1,1.30,6.5,0.5,2.0,0.0,POINT (-73.99154199999998 40.738918),POINT (-73.99154 40.73892),114,10011,New York,Manhattan
50,1,1.50,7.0,0.5,1.6,0.0,POINT (-73.991134 40.739729),POINT (-73.99113 40.73973),114,10011,New York,Manhattan
66,1,4.20,16.0,0.5,3.4,0.0,POINT (-74.005286 40.742399),POINT (-74.00529 40.74240),114,10011,New York,Manhattan
73,2,2.90,10.5,0.5,2.3,0.0,POINT (-74.007857 40.742873),POINT (-74.00786 40.74287),114,10011,New York,Manhattan
...,...,...,...,...,...,...,...,...,...,...,...,...
13328884,1,0.80,7.5,0.5,0.0,0.0,POINT (-73.892965 40.558363),POINT (-73.89297 40.55836),189,11697,Breezy Point,Queens
13331722,1,0.90,5.0,0.5,0.0,0.0,POINT (-73.892965 40.558363),POINT (-73.89297 40.55836),189,11697,Breezy Point,Queens
13334333,1,2.30,9.0,0.5,0.0,0.0,POINT (-73.892965 40.558363),POINT (-73.89297 40.55836),189,11697,Breezy Point,Queens
13360917,1,1.90,9.5,0.5,0.0,0.0,POINT (-73.892965 40.558363),POINT (-73.89297 40.55836),189,11697,Breezy Point,Queens


In [51]:
group = geo_processing.groupby(['postalCode']).describe()

In [53]:
group.to_csv('../../Data/yellow_tripdata_2014-1-start-processed.csv')

In [54]:
group

passenger_count                                                \
                      count      mean       std  min  25%  50%   75%  max   
postalCode                                                                  
00083              169941.0  1.782678  1.436789  1.0  1.0  1.0  2.00  6.0   
10001              844791.0  1.696533  1.361204  0.0  1.0  1.0  2.00  9.0   
10002              254397.0  1.750032  1.378956  0.0  1.0  1.0  2.00  9.0   
10003              774136.0  1.715104  1.377754  0.0  1.0  1.0  2.00  9.0   
10004               55284.0  1.724568  1.371994  1.0  1.0  1.0  2.00  8.0   
...                     ...       ...       ...  ...  ...  ...   ...  ...   
11691                   6.0  2.666667  2.250926  1.0  1.0  1.5  4.25  6.0   
11692                   2.0  1.000000  0.000000  1.0  1.0  1.0  1.00  1.0   
11693                   8.0  1.250000  0.462910  1.0  1.0  1.0  1.25  2.0   
11694                  16.0  1.125000  0.500000  1.0  1.0  1.0  1.00  3.0   
11697                  19.0  1.210526  0.917663  1.0  1.0  1.0  1.00  5.0   

            trip_distance            ...  tolls_amount        index_right  \
                    count      mean  ...           75%    max       count   
postalCode                           ...                                    
00083            169941.0  2.151628  ...           0.0  19.00    169941.0   
10001            844791.0  2.293111  ...           0.0  20.00    844791.0   
10002            254397.0  2.922630  ...           0.0  19.00    254397.0   
10003            774136.0  2.312567  ...           0.0  20.00    774136.0   
10004             55284.0  4.444352  ...           0.0  18.16     55284.0   
...                   ...       ...  ...           ...    ...         ...   
11691                 6.0  8.993333  ...           0.0   5.33         6.0   
11692                 2.0  1.400000  ...           0.0   0.00         2.0   
11693                 8.0  4.698750  ...           2.0   5.41         8.0   
11694                16.0  1.923750  ...           0.0   0.00        16.0   
11697                19.0  2.542105  ...           0.0   5.33        19.0   

                                                                     
                  mean       std    min    25%    50%    75%    max  
postalCode                                                           
00083        82.000000  0.000000   82.0   82.0   82.0   82.0   82.0  
10001       113.000000  0.000000  113.0  113.0  113.0  113.0  113.0  
10002       123.000000  0.000000  123.0  123.0  123.0  123.0  123.0  
10003       121.000000  0.000000  121.0  121.0  121.0  121.0  121.0  
10004       138.001519  0.101708  138.0  138.0  138.0  138.0  146.0  
...                ...       ...    ...    ...    ...    ...    ...  
11691       178.000000  0.000000  178.0  178.0  178.0  178.0  178.0  
11692       182.000000  0.000000  182.0  182.0  182.0  182.0  182.0  
11693       178.000000  8.280787  168.0  168.0  184.0  184.0  184.0  
11694       186.000000  0.000000  186.0  186.0  186.0  186.0  186.0  
11697       189.000000  0.000000  189.0  189.0  189.0  189.0  189.0  

[245 rows x 56 columns]